In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
def load_df(csv_path='../Data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../Data"))

['sample_submission.csv', 'test.csv', 'test_flattened.csv', 'train.csv', 'train_flattened.csv']


In [3]:
def load_df(csv_path='../Data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)

    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))

    return df

In [4]:
%%time
df_small = load_df(nrows=20000)

Wall time: 1.36 s


In [5]:
%%time
df_train = load_df()
df_test = load_df("../Data/test.csv")

Wall time: 1min 44s


In [6]:
%%time
df_train.to_csv("../Data/train_flattened.csv", index=False)
df_test.to_csv("../Data/test_flattened.csv", index=False)

Wall time: 1min 12s
